# Enrich Using Standard Geographies

Starting off, we import a few required Python resources. While there are quite a few in there, the import of note is `from arcgis.geoenrichment import Country, get_countries`. We are going to use this object and method to discover and perform our analysis.

In [33]:
import os
from pathlib import Path

from arcgis.features import GeoAccessor
from arcgis.geoenrichment import Country
from arcgis.gis import GIS
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer

Next, we need some test data to work with. Hence, here we are accessing two files with pickled exports from Spatially Enabled Pandas Data Frames. One is of postal codes (zip codes) in Portland, Oregon, and the other is for block groups in Portland.

In [26]:
# paths to input data
dir_prj = Path.cwd().parent
dir_data = dir_prj/'data'
dir_raw = dir_data/'raw'

# import the two preprocessors from the examples
sys.path.append(str(dir_prj/'src'))
from ba_samples.preprocessing import EnrichStandardGeography, KeepOnlyEnrichColumns

# specifically, the data being used for this example - pickled dataframes
postal_codes_pth = dir_raw/'postal_codes.pkl'
block_groups_pth = dir_raw/'block_groups.pkl'

For the first example, we are going to get a list of standard geography codes to use from the postal code data, just a list of zip codes.

In [27]:
postal_code_df = pd.read_pickle(postal_codes_pth)
postal_code_lst = list(postal_code_df['ID'])

print(postal_code_lst)

['83801', '83803', '83810', '83814', '83815', '83833', '83835', '83842', '83854', '83858', '83861', '83869', '83876']


For the latter, we are going to use the Data Frame directly, and tell the `enrich` function which column to look in when we invoke the function.

In [28]:
block_groups_df = pd.read_pickle(block_groups_pth)

block_groups_df.info()
block_groups_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       66 non-null     object
 1   NAME     66 non-null     object
 2   IDField  66 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


,ID,NAME,IDField
0,160550001001,160550001.001,17660
1,160550003011,160550003.011,17660
2,160550003012,160550003.012,17660
3,160550003021,160550003.021,17660
4,160550004011,160550004.011,17660


Now, we are going to need a connection to ArcGIS Online to demonstrate the abiliy to use ArcGIS Online for geoenrichment. This is accomplished by instantiating a `GIS` object instance with valid credentials read from environment variables.

In [29]:
gis_local = GIS('pro')

## Standard Geography From a List

To enrich, we start by creating a `Country` object instance. As part of the constructor, we need to tell the object what Business Analyst source to use in the `gis` parameter. In this case, we are telling the object to use ArcGIS Online.

In [30]:
usa_local = Country('usa', gis=gis_local)

usa_local

<Country - United States 2021 ('local')>

Next, we need to get some enrich variables to use. We can discover what is available using the `enrich_variables` property of the country object to retrieve a Pandas Data Frame of variables available for the country. From these tens of thousands of variables, we can prune this down to a manageable subset.

In [31]:
ev = usa_local.enrich_variables
kv = ev[
    (ev.data_collection.str.lower().str.contains('key'))
    & (ev.name.str.lower().str.endswith('cy'))
].reset_index(drop=True)

kv

,name,alias,data_collection,enrich_name,enrich_field_name
0,TOTPOP_CY,2021 Total Population,KeyUSFacts,KeyUSFacts.TOTPOP_CY,KeyUSFacts_TOTPOP_CY
1,GQPOP_CY,2021 Group Quarters Population,KeyUSFacts,KeyUSFacts.GQPOP_CY,KeyUSFacts_GQPOP_CY
2,DIVINDX_CY,2021 Diversity Index,KeyUSFacts,KeyUSFacts.DIVINDX_CY,KeyUSFacts_DIVINDX_CY
3,TOTHH_CY,2021 Total Households,KeyUSFacts,KeyUSFacts.TOTHH_CY,KeyUSFacts_TOTHH_CY
4,AVGHHSZ_CY,2021 Average Household Size,KeyUSFacts,KeyUSFacts.AVGHHSZ_CY,KeyUSFacts_AVGHHSZ_CY
5,MEDHINC_CY,2021 Median Household Income,KeyUSFacts,KeyUSFacts.MEDHINC_CY,KeyUSFacts_MEDHINC_CY
6,AVGHINC_CY,2021 Average Household Income,KeyUSFacts,KeyUSFacts.AVGHINC_CY,KeyUSFacts_AVGHINC_CY
7,PCI_CY,2021 Per Capita Income,KeyUSFacts,KeyUSFacts.PCI_CY,KeyUSFacts_PCI_CY
8,TOTHU_CY,2021 Total Housing Units,KeyUSFacts,KeyUSFacts.TOTHU_CY,KeyUSFacts_TOTHU_CY
9,OWNER_CY,2021 Owner Occupied HUs,KeyUSFacts,KeyUSFacts.OWNER_CY,KeyUSFacts_OWNER_CY


Finally, we are ready to enrich the postal codes using the resources above.

In [32]:
pipe = make_pipeline(
    EnrichStandardGeography(usa_local, enrich_variables=kv, standard_geography_level='zip5', return_geometry=False),
    KeepOnlyEnrichColumns(usa_local, id_column='id_field', keep_geometry=False)
)

trn_df = pipe.fit_transform(postal_code_lst)

trn_df.info()
trn_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 83801 to 83876
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   keyusfacts_totpop_cy   13 non-null     float64
 1   keyusfacts_gqpop_cy    13 non-null     float64
 2   keyusfacts_divindx_cy  13 non-null     float64
 3   keyusfacts_tothh_cy    13 non-null     float64
 4   keyusfacts_avghhsz_cy  13 non-null     float64
 5   keyusfacts_medhinc_cy  13 non-null     float64
 6   keyusfacts_avghinc_cy  13 non-null     float64
 7   keyusfacts_pci_cy      13 non-null     float64
 8   keyusfacts_tothu_cy    13 non-null     float64
 9   keyusfacts_owner_cy    13 non-null     float64
 10  keyusfacts_renter_cy   13 non-null     float64
 11  keyusfacts_vacant_cy   13 non-null     float64
 12  keyusfacts_medval_cy   13 non-null     float64
 13  keyusfacts_avgval_cy   13 non-null     float64
 14  keyusfacts_popgrw10cy  13 non-null     float64
 15  keyusf

,keyusfacts_totpop_cy,keyusfacts_gqpop_cy,keyusfacts_divindx_cy,keyusfacts_tothh_cy,keyusfacts_avghhsz_cy,keyusfacts_medhinc_cy,keyusfacts_avghinc_cy,keyusfacts_pci_cy,keyusfacts_tothu_cy,keyusfacts_owner_cy,keyusfacts_renter_cy,keyusfacts_vacant_cy,keyusfacts_medval_cy,keyusfacts_avgval_cy,keyusfacts_popgrw10cy,keyusfacts_hhgrw10cy,keyusfacts_famgrw10cy,keyusfacts_dpop_cy,keyusfacts_dpopwrk_cy,keyusfacts_dpopres_cy
id_field,,,,,,,,,,,,,,,,,,,,
83801,8497.0,0.0,13.6,3142.0,2.70,62413.0,78746.0,30156.0,3563.0,2660.0,482.0,421.0,382542.0,391184.0,2.07,2.09,1.92,6502.0,1962.0,4540.0
83803,656.0,0.0,10.5,341.0,1.92,43344.0,61732.0,29472.0,659.0,264.0,76.0,318.0,391333.0,403113.0,1.76,1.80,1.56,499.0,145.0,354.0
83810,1090.0,4.0,11.7,479.0,2.27,60088.0,89006.0,38340.0,579.0,406.0,72.0,100.0,437903.0,464383.0,0.94,0.96,0.76,762.0,246.0,516.0
83814,28546.0,609.0,21.2,12454.0,2.24,57487.0,80524.0,34960.0,15054.0,6855.0,5599.0,2600.0,377722.0,508589.0,1.86,1.91,1.72,38178.0,23011.0,15167.0
83815,38614.0,617.0,24.1,15247.0,2.49,56872.0,73431.0,29323.0,16393.0,9649.0,5598.0,1146.0,308198.0,374194.0,1.79,1.78,1.51,37345.0,15891.0,21454.0


In [42]:
pt = PowerTransformer(method='yeo-johnson')

arr = pt.fit_transform(trn_df)

pd.DataFrame(arr, columns=trn_df.columns, index=trn_df.index)

D:\Anaconda3\envs\geosaurus\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


,keyusfacts_totpop_cy,keyusfacts_gqpop_cy,keyusfacts_divindx_cy,keyusfacts_tothh_cy,keyusfacts_avghhsz_cy,keyusfacts_medhinc_cy,keyusfacts_avghinc_cy,keyusfacts_pci_cy,keyusfacts_tothu_cy,keyusfacts_owner_cy,keyusfacts_renter_cy,keyusfacts_vacant_cy,keyusfacts_medval_cy,keyusfacts_avgval_cy,keyusfacts_popgrw10cy,keyusfacts_hhgrw10cy,keyusfacts_famgrw10cy,keyusfacts_dpop_cy,keyusfacts_dpopwrk_cy,keyusfacts_dpopres_cy
id_field,,,,,,,,,,,,,,,,,,,,
83801,0.168550,-1.113505,-0.542176,0.112866,1.158966,0.240908,-0.183587,0.0,-0.046770,0.184312,-0.062410,-0.603029,0.064342,-0.460086,0.503606,0.507008,0.525388,0.077259,0.019262,0.136500
83803,-1.196066,-1.113505,-1.477190,-1.118763,-1.779359,-1.992848,-1.686528,0.0,-1.067416,-1.161138,-0.966999,-0.814751,0.173481,-0.335440,-0.085829,-0.061930,-0.182166,-1.208741,-1.222394,-1.204427
83810,-0.968891,-0.148353,-1.060640,-0.958105,-0.793368,-0.072514,1.246619,0.0,-1.130455,-0.954738,-0.991268,-1.439204,0.755506,0.264172,-1.541385,-1.598279,-1.628419,-1.028550,-0.993018,-1.043046
83814,1.041680,1.452413,0.638916,1.134051,-0.896162,-0.409677,0.032276,0.0,1.201580,0.938856,1.381679,1.747688,0.004606,0.660622,0.102147,0.151729,0.128315,1.290816,1.480525,1.019065
83815,1.287525,1.455045,0.899813,1.304346,0.085295,-0.487317,-0.756867,0.0,1.289198,1.247772,1.381563,0.435298,-0.848338,-0.642668,-0.029658,-0.100488,-0.277839,1.273683,1.240551,1.310222
83833,-0.758692,-1.113505,-1.124465,-0.707943,-1.182468,1.972745,1.408880,0.0,-0.371707,-0.699530,-0.737788,0.753802,2.302382,2.157659,1.717922,1.781086,1.750877,-0.648029,-0.545851,-0.693898
83835,0.907365,1.070701,0.509690,0.905488,0.366739,1.089360,1.227169,0.0,0.894612,0.995110,0.762944,0.932300,-0.046730,0.147940,0.121060,0.093194,0.108720,0.895797,0.904103,0.911211
83842,-1.781009,-1.113505,-1.328926,-1.811658,-0.576114,-0.213199,0.915263,0.0,-1.883583,-1.797255,-1.863280,-1.869657,0.767897,0.292604,-0.636757,-0.704894,-0.523486,-1.773304,-1.754636,-1.784556
83854,1.503007,1.068428,0.867499,1.463427,1.216565,-0.300930,-0.980865,0.0,1.501389,1.528723,1.356819,1.006537,-0.864607,-0.513656,1.534478,1.361912,1.368790,1.381439,1.261121,1.494684


## Standard Geography From a Data Frame

For the sake of efficiency, it is possible to provide the dataframe directly as input to the `enrich` method, and use the `standard_geography_id_column` parameter to specify the column. Also, in the following example, we are going to use a local source (ArcGIS Pro with Business Analyst and the USA data pack) and consolidate the steps above.

Previously, we loaded a dataframe from memory to use for enrichment consisting of all the block groups in Portland, OR. The values we will be using are located in the `ID` column.

In [13]:
block_groups_df.head()

,ID,NAME,IDField
0,160550001001,160550001.001,17660
1,160550003011,160550003.011,17660
2,160550003012,160550003.012,17660
3,160550003021,160550003.021,17660
4,160550004011,160550004.011,17660


Next, we create a `Country` object instance using the `'pro'` keyword to indicate using the local ArcGIS Pro instance.

In [14]:
usa_local = Country('usa', gis=GIS('pro'))

usa_local

<Country - United States 2021 ('local')>

Just as before, we can filter to just the key current year variables. Although in this case they are identical, since there are small variations in available variables between local and ArcGIS Online, I always apply the filter on variables from the same soure as I will be using for enricment.

In [15]:
ev = usa_local.enrich_variables
kv = ev[
    (ev.data_collection.str.lower().str.contains('key'))
    & (ev.name.str.lower().str.endswith('cy'))
].reset_index(drop=True)

kv

,name,alias,data_collection,enrich_name,enrich_field_name
0,TOTPOP_CY,2021 Total Population,KeyUSFacts,KeyUSFacts.TOTPOP_CY,KeyUSFacts_TOTPOP_CY
1,GQPOP_CY,2021 Group Quarters Population,KeyUSFacts,KeyUSFacts.GQPOP_CY,KeyUSFacts_GQPOP_CY
2,DIVINDX_CY,2021 Diversity Index,KeyUSFacts,KeyUSFacts.DIVINDX_CY,KeyUSFacts_DIVINDX_CY
3,TOTHH_CY,2021 Total Households,KeyUSFacts,KeyUSFacts.TOTHH_CY,KeyUSFacts_TOTHH_CY
4,AVGHHSZ_CY,2021 Average Household Size,KeyUSFacts,KeyUSFacts.AVGHHSZ_CY,KeyUSFacts_AVGHHSZ_CY
5,MEDHINC_CY,2021 Median Household Income,KeyUSFacts,KeyUSFacts.MEDHINC_CY,KeyUSFacts_MEDHINC_CY
6,AVGHINC_CY,2021 Average Household Income,KeyUSFacts,KeyUSFacts.AVGHINC_CY,KeyUSFacts_AVGHINC_CY
7,PCI_CY,2021 Per Capita Income,KeyUSFacts,KeyUSFacts.PCI_CY,KeyUSFacts_PCI_CY
8,TOTHU_CY,2021 Total Housing Units,KeyUSFacts,KeyUSFacts.TOTHU_CY,KeyUSFacts_TOTHU_CY
9,OWNER_CY,2021 Owner Occupied HUs,KeyUSFacts,KeyUSFacts.OWNER_CY,KeyUSFacts_OWNER_CY


Just double checking, we can quickly see the correct value we need to use for United States Census Block Groups is `block_groups`.

In [16]:
usa_local.levels

,level_name,alias,level_id,id_field,name_field,singular_name,plural_name,admin_level
0,block_groups,Block Groups,US.BlockGroups,ID,NAME,Block Group,Block Groups,Admin11
1,tracts,Census Tracts,US.Tracts,ID,NAME,Census Tract,Census Tracts,Admin10
2,places,Cities and Towns (Places),US.Places,ID,NAME,Place,Places,Admin9
3,zip5,ZIP Codes,US.ZIP5,ID,NAME,ZIP Code,ZIP Codes,Admin4
4,csd,County Subdivisions,US.CSD,ID,NAME,County Subdivision,County Subdivisions,Admin7
5,counties,Counties,US.Counties,ID,NAME,County,Counties,Admin3
6,cbsa,CBSAs,US.CBSA,ID,NAME,CBSA,CBSAs,Admin5
7,cd,Congressional Districts,US.CD,ID,NAME,Congressional District,Congressional Districts,Admin8
8,dma,DMAs,US.DMA,ID,NAME,DMA,DMAs,Admin6
9,states,States,US.States,ID,NAME,State,States,Admin2


Now, we are ready to enrich using ArcGIS Pro with Business Analyst and the USA data pack.

In [ ]:
bg_enrich_df = usa_local.enrich(block_groups_df, enrich_variables=kv, standard_geography_level='block_groups', standard_geography_id_column='ID')

bg_enrich_df.info()
bg_enrich_df.head()

## Consolidating into a Function

All of the above steps, if consolidated into a succicent function only containing the logic necessary to run, can look like the following. Also, although some of the imports below are redundant, I include everything needed in this cell soy you can easily copy this and modify it to suit your needs.

In [16]:
from typing import Iterable, Optional, Union

from arcgis.gis import GIS
from arcgis.geoenrichment import Country
import pandas as pd

def enrich_std_geo(
    country: Country, 
    geographies: Iterable, 
    std_geo_lvl: str, 
    enrich_vars: Optional[Iterable] = None, 
    std_geo_col: Optional[str] = None
) -> pd.DataFrame:
    
    # if no enrich variables provided, get the current year key variables
    if enrich_vars is None:
        enrich_vars = country.enrich_variables[
            (country.enrich_variables.data_collection.str.lower().str.contains('key'))
            & (country.enrich_variables.name.str.lower().str.endswith('cy'))
        ].reset_index(drop=True)
        
    # invoke enrich method against standard geographies
    enrich_df = country.enrich(geographies, 
                               enrich_variables=enrich_vars, 
                               standard_geography_level=std_geo_lvl, 
                               standard_geography_id_column=std_geo_col)
    
    return enrich_df

## Switching Sources

Using the above function we can easily accomplish all the steps detailed above in one succicent step. Based on this, we can quickly switch between sources by swapping out the `Country` instance. As is evident below, switching sources requires little more than changing the country object. Everything else works the same, and _this is the point_. This API is designed to enable you to easily work with Business Analyst data.

In [17]:
local_fn_enrich_df = enrich_std_geo(usa_local, geographies=postal_code_lst, std_geo_lvl='zip5')

local_fn_enrich_df.info()
local_fn_enrich_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id_field               13 non-null     object  
 1   area_desc              13 non-null     object  
 2   ta_desc                13 non-null     object  
 3   names                  13 non-null     object  
 4   has_data               13 non-null     int64   
 5   aggregation_method     13 non-null     object  
 6   keyusfacts_totpop_cy   13 non-null     float64 
 7   keyusfacts_gqpop_cy    13 non-null     float64 
 8   keyusfacts_divindx_cy  13 non-null     float64 
 9   keyusfacts_tothh_cy    13 non-null     float64 
 10  keyusfacts_avghhsz_cy  13 non-null     float64 
 11  keyusfacts_medhinc_cy  13 non-null     float64 
 12  keyusfacts_avghinc_cy  13 non-null     float64 
 13  keyusfacts_pci_cy      13 non-null     float64 
 14  keyusfacts_tothu_cy    13 non-null     float

,id_field,area_desc,ta_desc,names,has_data,aggregation_method,keyusfacts_totpop_cy,keyusfacts_gqpop_cy,keyusfacts_divindx_cy,keyusfacts_tothh_cy,...,keyusfacts_vacant_cy,keyusfacts_medval_cy,keyusfacts_avgval_cy,keyusfacts_popgrw10cy,keyusfacts_hhgrw10cy,keyusfacts_famgrw10cy,keyusfacts_dpop_cy,keyusfacts_dpopwrk_cy,keyusfacts_dpopres_cy,SHAPE
0,83801,83801,TA from geography Layer: 83801,Athol,1,BlockApportionment:US.BlockGroups;PointsLayer:...,8497.0,0.0,13.6,3142.0,...,421.0,382542.0,391184.0,2.07,2.09,1.92,6502.0,1962.0,4540.0,"{""rings"": [[[-116.69531754199994, 48.075561486..."
1,83803,83803,TA from geography Layer: 83803,Bayview,1,BlockApportionment:US.BlockGroups;PointsLayer:...,656.0,0.0,10.5,341.0,...,318.0,391333.0,403113.0,1.76,1.80,1.56,499.0,145.0,354.0,"{""rings"": [[[-116.49664843299996, 47.991467237..."
2,83810,83810,TA from geography Layer: 83810,Cataldo,1,BlockApportionment:US.BlockGroups;PointsLayer:...,1090.0,4.0,11.7,479.0,...,100.0,437903.0,464383.0,0.94,0.96,0.76,762.0,246.0,516.0,"{""rings"": [[[-116.40476999999998, 47.612660000..."
3,83814,83814,TA from geography Layer: 83814,Coeur D Alene,1,BlockApportionment:US.BlockGroups;PointsLayer:...,28546.0,609.0,21.2,12454.0,...,2600.0,377722.0,508589.0,1.86,1.91,1.72,38178.0,23011.0,15167.0,"{""rings"": [[[-116.58028267699996, 47.744349415..."
4,83815,83815,TA from geography Layer: 83815,Coeur D Alene,1,BlockApportionment:US.BlockGroups;PointsLayer:...,38614.0,617.0,24.1,15247.0,...,1146.0,308198.0,374194.0,1.79,1.78,1.51,37345.0,15891.0,21454.0,"{""rings"": [[[-116.69478052499994, 47.725640161..."


In [18]:
agol_fn_enrich_df = enrich_std_geo(usa_agol, geographies=postal_code_lst, std_geo_lvl='zip5')

agol_fn_enrich_df.info()
agol_fn_enrich_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 29 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   std_geography_level                13 non-null     object  
 1   std_geography_name                 13 non-null     object  
 2   std_geography_id                   13 non-null     object  
 3   source_country                     13 non-null     object  
 4   aggregation_method                 13 non-null     object  
 5   population_to_polygon_size_rating  13 non-null     float64 
 6   apportionment_confidence           13 non-null     float64 
 7   has_data                           13 non-null     int64   
 8   totpop_cy                          13 non-null     int64   
 9   gqpop_cy                           13 non-null     int64   
 10  divindx_cy                         13 non-null     float64 
 11  tothh_cy                           13 non-null 

,std_geography_level,std_geography_name,std_geography_id,source_country,aggregation_method,population_to_polygon_size_rating,apportionment_confidence,has_data,totpop_cy,gqpop_cy,...,vacant_cy,medval_cy,avgval_cy,popgrw10_cy,hhgrw10_cy,famgrw10_cy,dpop_cy,dpopwrk_cy,dpopres_cy,SHAPE
0,US.ZIP5,Athol,83801,US,Query:US.ZIP5,2.191,2.576,1,8497,0,...,421,382542,391184,2.07,2.09,1.92,6502,1962,4540,"{""rings"": [[[-116.69531754168734, 48.075561485..."
1,US.ZIP5,Bayview,83803,US,Query:US.ZIP5,2.191,2.576,1,656,0,...,318,391333,403113,1.76,1.80,1.56,499,145,354,"{""rings"": [[[-116.49664843313857, 47.991467236..."
2,US.ZIP5,Cataldo,83810,US,Query:US.ZIP5,2.191,2.576,1,1090,4,...,100,439516,465172,0.94,0.96,0.76,762,246,516,"{""rings"": [[[-116.40476999992698, 47.612659999..."
3,US.ZIP5,Coeur D Alene,83814,US,Query:US.ZIP5,2.191,2.576,1,28546,609,...,2600,377722,508589,1.86,1.91,1.72,38109,22942,15167,"{""rings"": [[[-116.58028267664488, 47.744349414..."
4,US.ZIP5,Coeur D Alene,83815,US,Query:US.ZIP5,2.191,2.576,1,38614,617,...,1146,308198,374194,1.79,1.78,1.51,37345,15891,21454,"{""rings"": [[[-116.69790643782714, 47.729762324..."
